notebook to test and find best hyperparameters for various models

In [3]:
import pickle
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import RandomizedSearchCV as RSCV

In [16]:
import os

pid = os.getpid()
print("PID: %i" % pid)

n_cpu = os.cpu_count()   # Number of CPUs assigned to this process
print("Number of CPUs in the system:", n_cpu)

# we won't use all the available cpu's for this script 
n_jobs = n_cpu - 2 # The number of tasks to run in parallel

# Control which CPUs are made available for this script
cpu_arg = ''.join([str(ci) + ',' for ci in list(range(n_jobs))])[:-1]
cmd = 'taskset -cp %s %i' % (cpu_arg, pid)
print("executing command '%s' ..." % cmd)
os.system(cmd)

PID: 25168
Number of CPUs in the system: 16
executing command 'taskset -cp 0,1,2,3,4,5,6,7,8,9,10,11,12,13 25168' ...


1

In [4]:
# path to cleaned data file
trainFilePath = 'data/data_train.pkl'

# Read already prepared and saved train datasets
with open(trainFilePath, 'rb') as f:
    data_train = pickle.load(f)

# Create dummy variables for categorical data
data_train = pd.get_dummies(data_train, columns=['clinical_stage', 'biopsy_gleason_gg', 'pathological_gleason_gg',
                                'pathologic_stage', 'lni', 'surgical_margin_status', 'persistent_psa',
                                'TRYSgrupes', 'PLNDO1'])

In [5]:
"""
Explodes the provided "df" dataset based on provided survival column "time" and
clips the data to be in a range [min_time; max_time] (). A new discrete survival column
will be created with name set as variable "time_discrete". "cum_event" boolean determines
if cumulative event column will be created or no.

clip(lower, upper) function will help us create a new discrete survival time column. 
If we specify lower=1 and upper=200, patients who experienced event earlier than 200th 
month will only have records till their event, on other side, if a patient survived past 
200th month, we will clip this information and will only keep information about him til 200th month.
Another example, if we specify lower=140 and upper=200, and if the person experienced event 
at 100th month, we will create records for him till 140th (lower boundary) month.
"""
def explode_data(df,max_time,time,target_column,min_time=1,
                 time_discrete='survival_time_discrete',cum_event=False):

    target_column_discrete = target_column + '_discrete'

    # We create a new time column and clip the data by provided min and max survival times
    df[time_discrete] = df[time].clip(min_time,max_time).apply(range)

    # Exploding the dataset with the created range value in new time column
    data_exploded = df.explode(time_discrete)
    data_exploded.reset_index(drop=True, inplace=True)

    # New column starts at 0, we'll increase each value by 1
    data_exploded[time_discrete] = pd.to_numeric(data_exploded[time_discrete]) + 1

    # New event column, which will indicate the last event date
    data_exploded[target_column_discrete] = (data_exploded[time_discrete] >= data_exploded[time]) * pd.to_numeric(data_exploded[target_column])
    
    if cum_event == True:
        target_column_cumulative = target_column + '_cumulative'

        # Create new event column with duplicated event values from discrete column
        data_exploded[target_column_cumulative] = data_exploded[target_column_discrete]
        
        # For cumulative events, after end_time we will have NA values, we'll replace those with event indicator
        after_survival_time = data_exploded[time_discrete] > data_exploded[time]
        data_exploded.loc[after_survival_time, target_column_discrete] = -1
        data_exploded[target_column_discrete] = data_exploded[target_column_discrete].replace(-1,np.NaN)
        data_exploded.loc[(after_survival_time & (data_exploded[target_column]==0)), target_column_cumulative] = -1
        data_exploded[target_column_cumulative] = data_exploded[target_column_cumulative].replace(-1,np.NaN)

    return data_exploded

## Random Forest

### Cancer specific mortality

In [11]:
target_column = 'cancer_specific_mortality'
max_time = 200

# mts and bcr have different survival months columns
match target_column:
    case 'mts':
        time = 'survival_months_mts'
    case 'bcr':
        time = 'survival_months_bcr'
    case _:
        time = 'survival_months'

target_column_discrete = target_column + '_discrete'

# List of columns names which will be dropped from feature set before fitting the model
target_columns = ['cancer_specific_mortality', 'death_from_other_causes', 'bcr', 'mts']

# Explode the dataset
df_train_copy_exploded = explode_data(data_train.copy(), min_time=1, max_time=max_time, time=time, target_column=target_column)

# Drop targets/features from feature set
x_columns_to_drop = [target_column_discrete, 'survival_months', 'survival_months_bcr', 'survival_months_mts', 'patient_id']
x_columns_to_drop.extend(target_columns)
X_train = df_train_copy_exploded.drop(x_columns_to_drop, axis=1)    
y_train = df_train_copy_exploded[target_column_discrete]

In [13]:
X_train.columns

Index(['age', 'psa', 'biopsy_gleason', 'pathologic_gleason',
       'clinical_stage_1', 'clinical_stage_2', 'clinical_stage_3',
       'biopsy_gleason_gg_1', 'biopsy_gleason_gg_2', 'biopsy_gleason_gg_3',
       'biopsy_gleason_gg_4', 'biopsy_gleason_gg_5',
       'pathological_gleason_gg_1', 'pathological_gleason_gg_2',
       'pathological_gleason_gg_3', 'pathological_gleason_gg_4',
       'pathological_gleason_gg_5', 'pathologic_stage_0', 'pathologic_stage_1',
       'pathologic_stage_2', 'lni_0.0', 'lni_1.0', 'lni_unknown',
       'surgical_margin_status_0', 'surgical_margin_status_1',
       'persistent_psa_0', 'persistent_psa_1', 'TRYSgrupes_0', 'TRYSgrupes_1',
       'TRYSgrupes_2', 'PLNDO1_0', 'PLNDO1_1', 'survival_time_discrete'],
      dtype='object')

In [18]:
param_grid = {'n_estimators':np.arange(150,700,50),
              'max_features':np.arange(0.1, 1, 0.1),
              'max_depth': [3, 5, 7, 9],
              'max_samples': [0.3, 0.5, 0.8]}

rf_random = RSCV(RandomForestClassifier(), param_grid, n_iter=15, 
             random_state=0, verbose=2, scoring='roc_auc', n_jobs=n_jobs)
model_rf = rf_random.fit(X_train, y_train)
model_rf_best = model_rf.best_estimator_

Fitting 5 folds for each of 15 candidates, totalling 75 fits


In [20]:
# Best parameters
model_rf.best_params_

{'n_estimators': 350,
 'max_samples': 0.8,
 'max_features': 0.7000000000000001,
 'max_depth': 3}